In [1]:
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import sklearn.feature_extraction.text as sk_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator, RegressorMixin
from wordcloud import WordCloud
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from collections import Counter
import nltk
from sklearn.metrics import roc_curve
import math
from sklearn.metrics import precision_recall_curve
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

Let us import the debate speech data and specify a list unwated words that must be romoved from the speech data. These words are moderator names and states names that are meaningless to our model. We will also define a list of election terms with their start and end dates.

In [2]:
speech_data = pd.read_csv('finaldata_party.csv')

election_term = [1992,1996,2000,2004,2008,2012,2016,2020]
start_dates = ['1992-01-01','1996-01-01','2000-01-01','2004-01-01','2008-01-01','2012-01-01','2016-01-01','2020-01-01']
end_dates = ['1996-01-01','2000-01-01','2004-01-01','2008-01-01','2012-01-01','2016-01-01','2022-01-01','2022-01-01']

moderators = ['Dorothy Ridings', 'Sander Vanocur', 'John Mashek', 'Jack White',
       'Norma Quarles', 'Robert Boyd', 'Jim Lehrer', 'Ann Compton',
       'Moderator', 'Martha Raddatz', 'Bill Shadel', 'Frank McGee',
       'Charles Van Fremd', 'Douglass Cater', 'Rosco Drummond',
       'Bernard Shaw', 'Margaret Warner', 'Andrea Mitchell', 'Hal Bruno',
       'Bob Schieffer', 'Unknown', 'Carol Simpson', 'Question',
       'Kate Kelly', 'Gary Johnson', 'Bernie Sanders', 'Howard Smith',
       'Walter Cronkite', 'Candy Crowley', 'Audience', 'Helen Thomas',
       'Gene Gibbons', 'Chris Wallace', 'Quincy Howe', 'Frank Singiser',
       'John Edwards', 'John Chancellor', 'Edwin Newman', 'Georgie Geyer',
       'Marvin Kalb', 'Morton Kondracke', 'Barbara Walters',
       'Joseph Kraft', 'Robert Maynard', 'Jack Nelson', 'Marvin Stone',
       'Harry Ellis', 'William Hilliard', 'Elaine Quijano',
       'Judy Woodruff', 'Jon Margolis', 'Tom Brokaw', 'Brit Hume',
       'Gwen Ifill', 'Pauline Frederick', 'Max Frankel', 'Henry Trewitt',
       'Richard Valeriani', 'Paul Niven', 'Edward Morgan', 'Alvin Spivak',
       'Harold Levy', 'James Wieghart', 'Dianne Sawyer', 'Fred Barnes',
       'Charles Gibson', 'Anderson Cooper', 'John Anderson',
       'Bill Moyers', 'Carol Loomis', 'Daniel Greenberg',
       'Charles Corddry', 'Lee May', 'Washington Post', 'Bryant Quinn',
       'New York Times', 'Soma Golden', 'Frank Reynolds', 'James Gannon',
       'Elizabeth Drew', 'Anne Groer', 'Peter Jennings', 'Bob Fleming',
       'Stuart Novins', 'Charles Warren', 'Voiceover', 'Lester Holt',
       'Franchesca Ramsey', 'Connor Franta', 'Marques Brownlee',
       'Dana Bash', 'Don Lemon', 'Juan Carlos Lopez', 'David Muir',
       'Neil Levesque', 'Josh McElveen', 'Rachel Maddow',
       'Kristen Welker', 'Ashley Parker', 'Erin Burnett', 'Marc Lacey',
       'Jake Tapper', 'Marrianne Williamson', 'John King',
       'Nia-Malika Henderson', 'Michael Bennet', 'crowd',
       'George Stephanopoulos', 'Jorge Ramos', 'Lindsey Davis',
       'Wolf Blitzer', 'Abby Phillip', 'Brianne Pfannenstiel',
       'Hallie Jackson', 'Chuck Todd', 'Vanessa Hauc', 'Jon Ralston',
       'Protestors', 'Tim Alberta', 'Amna Nawaz', 'Yamiche Alcindor',
       'Savannah Guthrie', 'Jose Diaz', 'Steve Kornacki', 'Lindsey David',
       'Monica Hernandez', 'Adam Sexton', 'Linsey Davis', 'Devin Dwyer',
       'Rachel Scott', 'Announcer', 'Steve King', "Norah O'Donnell",
       'Candidate', 'Bill Whitaker', 'Major Garrett', 'Margaret Brennan',
       'Ilia Caldern', 'Sanjay Gupta', 'Amy Langenfeld', 'David Gergen',
       'Erick Erickson', 'Michelle Bachmann', 'Jim Demint',
       'Unidentified', 'Resident', 'Bret Baier', 'Gerald Seib',
       'Kelly Evans', 'Brian Williams', 'Shannon Bream', 'Rick Scott',
       'Maria Bartiromo', 'John Harwood', 'Jim Cramer', 'Steve Liesman',
       'Narrator', 'Doug Oberhelman', 'Rick Santelli', 'Sharon Epperson',
       'Matt Strawn', 'Protester', 'Hugh Hewitt', 'Susan Page ']

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

states = list(states.values())
moderator_names = []
for w in moderators:
    for a in w.split():
        moderator_names.append(a.lower())

states_names = []
for w in states:
    for a in w.split():
        states_names.append(a.lower())

unwanted_names = moderator_names + states_names


Now we will define few functions that will be used in our final Pipeline. 

In [3]:
class Custom_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self,min_date,max_date,type):
        '''
        Inputs:
        min_date: Select the start date, format: "YYYY-MM-DD",(if None selects all dates)
        max_date: Select the end date, format: "YYYY-MM-DD",(if None selects all dates)
        type: List of debate types, e.g ['VP','Pres','Dem','Rep'],(if None all types will be considered)
        
        Output:
        Debate Date, Debate Type selected data frame which contains only the 'Democratic' and 'Republican' party debates
        with targets 0 for 'Republican' and 1 for 'Democratic'
        '''
        self.min_date = min_date
        self.max_date = max_date
        self.type = type
        
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if self.min_date!=None and self.max_date!=None:
            X['date'] = pd.to_datetime(X['date'])
            date_mask = (X['date'] > self.min_date) & (X['date'] <= self.max_date)
            masked_data = X.loc[date_mask]
        else:
            masked_data = X
    
        if self.type!=None:
            masked1_data = masked_data.loc[masked_data['type'].isin(self.type)]
        else:
            masked1_data = masked_data
        masked2_data = masked1_data.loc[masked_data['party'].isin(['R','D'])]
        masked2_data.loc[masked2_data['party'] == 'R', 'party'] = 0
        masked2_data.loc[masked2_data['party'] == 'D', 'party'] = 1
        masked2_data['party'] = masked2_data['party'].astype(int)
        return masked2_data
        
def get_custom_stopwords(df_new,unwanted_names):
    all_Text = df_new['text']
    Tfidf = sk_text.TfidfVectorizer(lowercase=True,analyzer='word',stop_words='english',ngram_range=(1,1))
    Tfidf.fit_transform(all_Text)
    all_words = Tfidf.get_feature_names_out()
    numeric_words = []
    for w in all_words:
        if not w.isalpha():
           numeric_words.append(w)

    candidates = df_new['speaker'].unique()
    candidate_names = []
    for w in candidates:
        for a in w.split():
            candidate_names.append(a.lower())
        
    stop_words = stopwords.words('english')
    custom_stopwords = stop_words + numeric_words + candidate_names + unwanted_names
    return custom_stopwords

def lemmatizing_function(df, col_name, stopwords):
    lemmatizer = WordNetLemmatizer()
    # Input is the dataframe, the column name containing the text and the list of stopwords
    lemmata = []
    for obs in df[col_name]:
        # Tokenizing the string in the observation
        lemma = []
        words = word_tokenize(obs)
        # Iterating through the words
        for word in words:
            # Checking if not in stop words
            if word.lower() not in stopwords:
                w1 = lemmatizer.lemmatize(word.lower(), pos='n')
                w2 = lemmatizer.lemmatize(w1, pos = 'v')
                w3 = lemmatizer.lemmatize(w2, pos='a')
                lemma.append(w3)
                
        # Combining into a sentence
        new = ' '.join(lemma)
        # Appending to the list of observations
        lemmata.append(new)
    # Appending to the dataframe
    name = col_name + '_lemma'
    df[name] = lemmata
    return df

def transform_custom_text(custom,custom_stopwords):
    custom_test = pd.DataFrame(custom, columns=['text'])
    custom_test_lemmatized = lemmatizing_function(custom_test,'text',custom_stopwords)
    return custom_test_lemmatized

def political_sentiment_analysis_pipeline(speech_data,election_term,start_dates,end_dates,custom_text,unwanted_names,classifier):
    recall = {}
    precision = {}
    accuracy = {}
    f1score = {}
    recall_train = {}
    precision_train = {}
    accuracy_train = {}
    f1score_train = {}
    custom_thresholds = {}
    custom_recall = {}
    custom_precision = {}
    custom_accuracy = {}
    custom_f1score = {}
    democratic_probability = np.zeros((len(election_term),len(custom_text)))
    for i in range(0,len(start_dates)):
        term = election_term[i]
        start = start_dates[i]
        end = end_dates[i]
        Transformer = Custom_Transformer(min_date=start,max_date=end,type=['Pres','VP'])
        df_new = Transformer.fit_transform(speech_data)
        Tfidf = sk_text.TfidfVectorizer(lowercase=True,stop_words='english',analyzer='word',ngram_range=(1,2))
        custom_stopwords = get_custom_stopwords(df_new,unwanted_names)
        lemmatized_df = lemmatizing_function(df_new,'text',custom_stopwords)
        df_train,df_test = train_test_split(lemmatized_df.copy(),test_size=0.3, random_state=42,stratify=lemmatized_df['party'])
        X = df_train['text_lemma']
        y = df_train['party']
        X_test = df_test['text_lemma']
        y_test = df_test['party']
        Tfidf.fit_transform(df_train['text_lemma'])
        Party_classifier = Pipeline([("Tfidf vectorizer",Tfidf),("classifier",classifier)])
        Party_classifier.fit(X,y)
        y_predict = Party_classifier.predict(X_test)
        y_predict_prob = Party_classifier.predict_proba(X_test)[:,1]
        y_predict_train = Party_classifier.predict(X)
        y_predict_train_prob = Party_classifier.predict_proba(X)[:,1]
        test_metrics = [precision_score(y_test,y_predict), recall_score(y_test,y_predict), f1_score(y_test,y_predict), accuracy_score(y_test,y_predict)]
        train_metrics = [precision_score(y,y_predict_train), recall_score(y,y_predict_train), f1_score(y,y_predict_train), accuracy_score(y,y_predict_train)]
        recall[term] = recall_score(y_test,y_predict)
        precision[term] = precision_score(y_test,y_predict)
        f1score[term] = f1_score(y_test,y_predict)
        accuracy[term] = accuracy_score(y_test,y_predict)
        recall_train[term] = recall_score(y,y_predict_train)
        precision_train[term] = precision_score(y,y_predict_train)
        f1score_train[term] = f1_score(y,y_predict_train)
        accuracy_train[term] = accuracy_score(y,y_predict_train)
        custom_test_lemmatized = transform_custom_text(custom_text,custom_stopwords)
        precision_t, recall_t, thresholds_t = precision_recall_curve(y_test, y_predict_prob)
        fscore_t = (2 * precision_t * recall_t) / (precision_t + recall_t)
        ix_fscore = np.argmax(fscore_t)
        custom_thresholds[term] = thresholds_t[ix_fscore]
        custom_recall[term] = recall_t[ix_fscore]
        custom_precision[term] = precision_t[ix_fscore]
        custom_f1score[term] = fscore_t[ix_fscore]
        custom_predictions = []
        for p in y_predict_prob:
            if p >= thresholds_t[ix_fscore]:
               custom_predictions.append(1)
            else:
               custom_predictions.append(0)
        custom_accuracy[term] = accuracy_score(y_test,custom_predictions)
        y_custom = Party_classifier.predict_proba(custom_test_lemmatized['text_lemma'])[:,1]
        democratic_probability[i,:] = y_custom
    return democratic_probability, recall, recall_train, precision, precision_train, f1score, f1score_train, accuracy, accuracy_train, custom_thresholds, custom_recall, custom_precision, custom_f1score, custom_accuracy


The code will take in a list of custom words or text and output the probability of that word/text belonging to the Democratic party $P(D)$ as a function of time (from 1960-2020). Note that the data is only used for Republican and Democraatic party so probability of word belonging to Republicaan party $P(R) = 1 - P(D)$. 

In [4]:
custom_text = ['Pro-Life', 'Pro-Choice', 'Right to Choose',
              'Illegal Aliens', 'Illegal Immigrants', 'Undocumented Immigrants',
              'Social Security', 'White Supremacists', 'Strong Military']

MNB = MultinomialNB()
RF = RandomForestClassifier(n_estimators = 100, max_depth = 10)
democratic_probability,rc,rc_tr,pr,pr_tr,f1,f1_tr,acc,acc_tr,custom_thresholds, c_rc, c_pr, c_f1, c_acc = political_sentiment_analysis_pipeline(speech_data,election_term,start_dates,end_dates,custom_text,unwanted_names,MNB)


c:\users\mirad\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\mirad\AppData\Local\Temp/ipykernel_19236/2763065327.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masked2_data['party'] = masked2_data['party'].astype(int)


Lets plot the probability of the 9 text/words as a function of time.

In [5]:
custom_thresholds

{1992: 0.4236075367378914,
 1996: 0.4179299441951274,
 2000: 0.4436984207636946,
 2004: 0.517967698663595,
 2008: 0.5286218872758689,
 2012: 0.4656488549618322,
 2016: 0.39801278485180874,
 2020: 0.43889889892595413}

In [15]:
def probabilities_plotter(text, text_probs, years, prob_thresholds, num_cols, figsize):
    # Inputs are the list of texts, the matrix of text probabilities generated by model, 
    # the years, the probability thresholds, the number of columns and the figure size
    
    # Setting some defaults
    ft = 250
    lw = 15
    ltw = 20
    ltwm= 70
    pad_space=50
    lz=250
    
    # Extending the prob_thresholds and the text_probs to 4 observations per point
    x_axis = np.arange(years[0], years[-1]+4)
    dem_probs = np.repeat(text_probs, 4)
    thresholds = np.repeat(list(prob_thresholds.values()), 4)
    # Setting the number of columns
    num_rows = int(len(text) // num_cols) + 1
    # Setting the figure and the figure size
    f = plt.figure(figsize=figsize)
    f.subplots_adjust(hspace = 0.3)
    
    for i in range(len(text)):
        dem_probs = np.repeat(text_probs[:, i], 4)
        ax = plt.subplot(num_rows, num_cols, i+1)
        ax.step(x_axis, dem_probs, c='m', marker='_', linewidth=lw, where='post', label="Probability")
        ax.step(x_axis, thresholds, c='k', marker='_', linewidth=lw, where='post', label="Threshold")
        ax.fill_between(x_axis, 0, 1, where=dem_probs >= thresholds, facecolor = 'blue', alpha = 0.40)
        ax.fill_between(x_axis, 0, 1, where=dem_probs < thresholds, facecolor = 'red', alpha = 0.40)
        ax.set_ylabel('P(D|Text)', fontsize=ft)
        ax.set_xticks(np.arange(years[0], years[-1]+4, 4)) 
        ax.set_xlabel('Year', fontsize=ft)
        ax.set_title(text[i], fontsize=250)
        ax.minorticks_on()
        ax.set_ylim([0,1])
        ax.tick_params(which='major', length=ltwm, width=lw, direction='in', labelsize=lz,
                       axis='both', bottom=True, top=True, left=True, right=True, pad=pad_space)
        ax.tick_params(which='minor', length=ltw, width=lw, direction='in', labelsize=lz,
                       axis='both', bottom=True, top=True, left=True, right=True, pad=pad_space)
        for axis in ['top','bottom','left','right']:
                ax.spines[axis].set_linewidth(lw)
    
    f.legend(['Probabiliity', 'Thresholds'], fontsize=250, loc='right')
    plt.savefig('custom_text.pdf', bbox_inches='tight')
    plt.show()

In [16]:
probabilities_plotter(custom_text, democratic_probability, election_term, custom_thresholds, 3, (400,300))

In [8]:
f = plt.figure(figsize=(100,80))

ft = 150
lw = 18
ltw = 20
ltwm= 30
pad_space=50
lz=150

ax1 = f.add_subplot(221)
ax1.plot(election_term,list(rc.values()),c='g',linewidth=lw,label='Test')
ax1.plot(election_term,list(rc_tr.values()),c='orange',linewidth=lw,label='Train')
ax1.plot(election_term,list(c_rc.values()),c='r',linewidth=lw,label='Custom Test')
ax1.set_ylabel('Recall',fontsize=ft)
ax1.set_xlabel('time (years)',fontsize=ft)
ax1.legend(fontsize=ft,frameon=False)
ax1.minorticks_on()
ax1.set_ylim([0,1.2])
ax1.tick_params(which='major',length=ltw,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
ax1.tick_params(which='minor',length=ltwm,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
for axis in ['top','bottom','left','right']:
        ax1.spines[axis].set_linewidth(lw)

ax2 = f.add_subplot(222)
ax2.plot(election_term,list(pr.values()),c='g',linewidth=lw)
ax2.plot(election_term,list(pr_tr.values()),c='orange',linewidth=lw)
ax2.plot(election_term,list(c_pr.values()),c='r',linewidth=lw)
ax2.set_ylabel('Precision',fontsize=ft)
ax2.set_xlabel('time (years)',fontsize=ft)
ax2.minorticks_on()
ax2.set_ylim([0,1.2])
ax2.tick_params(which='major',length=ltw,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
ax2.tick_params(which='minor',length=ltwm,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
for axis in ['top','bottom','left','right']:
        ax2.spines[axis].set_linewidth(lw)

ax3 = f.add_subplot(223)
ax3.plot(election_term,list(f1.values()),c='g',linewidth=lw)
ax3.plot(election_term,list(f1_tr.values()),c='orange',linewidth=lw)
ax3.plot(election_term,list(c_f1.values()),c='r',linewidth=lw)
ax3.set_ylabel('F1 score',fontsize=ft)
ax3.set_xlabel('time (years)',fontsize=ft)
ax3.minorticks_on()
ax3.set_ylim([0,1.2])
ax3.tick_params(which='major',length=ltw,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
ax3.tick_params(which='minor',length=ltwm,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
for axis in ['top','bottom','left','right']:
        ax3.spines[axis].set_linewidth(lw)

ax4 = f.add_subplot(224)
ax4.plot(election_term,list(acc.values()),c='g',linewidth=lw)
ax4.plot(election_term,list(acc_tr.values()),c='orange',linewidth=lw)
ax4.plot(election_term,list(c_acc.values()),c='r',linewidth=lw)
ax4.set_ylabel('Accuracy',fontsize=ft)
ax4.set_xlabel('time (years)',fontsize=ft)
ax4.minorticks_on()
ax4.set_ylim([0,1.2])
ax4.tick_params(which='major',length=ltw,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
ax4.tick_params(which='minor',length=ltwm,width=lw,direction='in',labelsize=lz,axis='both',bottom=True,top=True,left=True,right=True,pad=pad_space)
for axis in ['top','bottom','left','right']:
        ax4.spines[axis].set_linewidth(lw)
        

f.tight_layout()
f.savefig('Metrics_MNB.pdf')
plt.close()

